In [1]:
import os
import glob
import random 
import itertools

import numpy as np
import pandas as pd

from pathlib import Path
from scipy import signal

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split, Subset

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_encoder = preprocessing.LabelEncoder()

In [2]:
class EchoStateNetwork(nn.Module):
    def __init__(self, model_params, dataset_params):
        super(EchoStateNetwork, self).__init__()
        
        self.reservoir_size = int(model_params["reservoir_size"])
        self.reservoir_weights_scale = float(model_params["reservoir_weights_scale"])
        
        self.input_size = int(model_params["input_size"])
        self.channel_size = int(model_params["channel_size"])
        self.input_weights_scale = float(model_params["input_weights_scale"])
        self.spectral_radius = float(model_params["spectral_radius"])
        self.density = float(model_params["reservoir_density"])
        self.leak_rate = float(model_params["leak_rate"])
        
        self.sequence_length = int(int(dataset_params["sequence_length"]) / int(dataset_params["slicing_size"]))
        # self.sequence_length = int(dataset_params["seq_end"]) - int(dataset_params["seq_start"])

        # リザバー結合行列 (ランダムに初期化)
        self.register_parameter("reservoir_weights", torch.nn.Parameter(torch.empty((self.reservoir_size, self.reservoir_size)).uniform_(-self.reservoir_weights_scale, self.reservoir_weights_scale).to(device), requires_grad=False))
        
        # リザバー入力行列 (ランダムに初期化)
        self.register_parameter("input_weights", torch.nn.Parameter(torch.empty((self.reservoir_size, self.input_size * self.channel_size)).uniform_(-self.input_weights_scale, self.input_weights_scale).to(device), requires_grad=False))
        
        

        #リザバー結合のスパース処理
        self.reservoir_weights_mask = torch.empty((self.reservoir_size, self.reservoir_size)).uniform_(0, 1)
        self.reservoir_weights_mask = torch.where(self.reservoir_weights_mask < self.density, torch.tensor(1), torch.tensor(0)).to(device)
        self.reservoir_weights *= self.reservoir_weights_mask
        
        #スペクトル半径の処理
        _, singular_values, _ = torch.svd(self.reservoir_weights)
        rho_reservoir_weights = torch.max(singular_values).item()
        self.reservoir_weights *= self.spectral_radius / rho_reservoir_weights
        
       #最終時刻における，リザバー状態ベクトル
        self.last_reservoir_state_matrix = torch.zeros(self.channel_size, self.reservoir_size).to(device)
    

    def forward(self, x):
        # print(x.shape)
        batch_size = x.size(0)
        sequence_length = x.size(2)
        # x_seq = x.view(batch_size, self.channel_size, self.input_size, sequence_length).to(device)

        # if self.last_reservoir_state_matrix is None or self.last_reservoir_state_matrix.size(0) != batch_size:
        #     self.last_reservoir_state_matrix = torch.zeros(
        #         batch_size, self.channel_size, self.reservoir_size, device=device
        #     )
        
        # 各シーケンスのバッチに対してリザバー状態を初期化
        self.reservoir_state_matrix = torch.zeros(batch_size, self.channel_size,  sequence_length, self.reservoir_size).to(device)

        self.last_reservoir_state_matrix = torch.zeros(
                batch_size, self.channel_size, self.reservoir_size, device=device
            )
        
        for t in range(sequence_length):
            input_at_t = torch.matmul(x[:, :, t, :].view(batch_size, -1), self.input_weights.t())
            input_at_t = input_at_t.unsqueeze(1)

            if t == 0:
                state_update = torch.matmul(self.last_reservoir_state_matrix, self.reservoir_weights)
            else:
                # print("11111")
                state_update = torch.matmul(self.reservoir_state_matrix[:, :, t-1, :], self.reservoir_weights)
                # print(f"state_update.shape {state_update.shape}")
            self.reservoir_state_matrix[:, :, t, :] = self.leak_rate * torch.tanh(input_at_t + state_update) + \
                                                    (1 - self.leak_rate) * self.reservoir_state_matrix[:, :, t-1, :]

        

        self.last_reservoir_state_matrix = self.reservoir_state_matrix[:, :, -1, :]
        return self.reservoir_state_matrix

    def reset_hidden_state(self):
        self.last_reservoir_state_matrix = torch.zeros(self.channel_size, self.reservoir_size, device=device)
        # pass
        # print("内部状態がリセットされました")
    
#リードアウト層
class ReadOut(nn.Module):
    def __init__(self, model_params, dataset_params):
        super(ReadOut, self).__init__()
        # self.reservoir_state_matrix_size = int(model_params["reservoir_size"]) + int(model_params["input_size"]) + 1
        self.reservoir_state_matrix_size = int(model_params["reservoir_size"])
        self.output_size = int(model_params["ReadOut_output_size"])
        self.batch_training = model_params["Batch_Training"]
        self.channel_size = int(model_params["channel_size"])
        
        self.sequence_length = int(int(dataset_params["sequence_length"]) / int(dataset_params["slicing_size"]))
        # self.sequence_length = int(dataset_params["seq_end"]) - int(dataset_params["seq_start"])
        
        self.readout_dense = nn.Linear(self.reservoir_state_matrix_size, self.output_size, bias=False)
        
        #線形回帰におけるバッチ学習を行うならば，リードアウト層を最急降下法による学習対象にしない
        if self.batch_training == True:
            self.readout_dense.weight.requires_grad = False
        else:
            None
            
        nn.init.xavier_uniform_(self.readout_dense.weight)
        
    def forward(self, x):
        # x shape: [batch_size, channel_size, input_size, sequence_length]
        # batch_size, channel_size, input_size, sequence_length = x.size()
        
        # Reshape x to apply the dense layer to each time step and channel independently
        # New shape: [batch_size * channel_size * sequence_length, input_size]
        # x_reshaped = x.permute(0, 1, 3, 2).contiguous().view(-1, input_size)
        
        # Apply the dense layer
        # Output shape: [batch_size * channel_size * sequence_length, output_size]
        # print(f"readout x shape{x.shape}")
        output = self.readout_dense(x)
        
        # Reshape the output back to the original form
        # Output shape: [batch_size, channel_size, output_size, sequence_length]
        # output = output.view(batch_size, channel_size, sequence_length, -1).permute(0, 1, 3, 2).contiguous()
        return output
    
    # リッジ回帰によるリードアウトの導出
    @staticmethod
    def ridge_regression(X, Y, alpha):
        # データ行列 X の形状を取得
        n, p = X.shape

        # 正則化項の行列を作成
        ridge_matrix = (alpha * torch.eye(n)).float().to(device)
        X = X.float()
        Y = Y.float()

        # リッジ回帰の係数を計算
        coefficients = torch.linalg.solve(torch.matmul(X, X.T) + ridge_matrix, torch.matmul(X, Y.T)).T

        return coefficients

    @staticmethod
    def ridge_regression_update(outputs, targets, model, alpha=0):
        with torch.no_grad():
            # リッジ回帰を用いて重みを求める
            # モデルのパラメータを更新
            new_weights = ReadOut.ridge_regression(outputs.squeeze(), targets.squeeze(), alpha)

            # モデルのパラメータを更新
            model.ReadOut.readout_dense.weight.copy_(new_weights)
        
        return None
    #それぞれのモデルパラメータ候補を辞書に格納する
    @staticmethod
    def model_params_candinate(model_params):
        model_params_combinations = list(itertools.product(*model_params.values()))
        param_dicts = [dict(zip(model_params.keys(), combination)) for combination in model_params_combinations]
        
        return param_dicts

    #モデル構造を辞書型に格納
    @staticmethod
    def model_sturcture_dict(model):
        layers_dict = {}
        for name, module in model.named_modules():
            layers_dict[name] = {
                'type': type(module).__name__,
                'parameters': {p: getattr(module, p) for p in module.__dict__ if not p.startswith('_')}
            }
        
        #モデル名と初期の引数は削除
        del(layers_dict[''])
        
        return layers_dict

class ESN(nn.Module):
    def __init__(self, model_params, training_params, dataset_params):
        super(ESN, self).__init__()
        self.ESN = EchoStateNetwork(model_params, dataset_params)
        self.ReadOut = ReadOut(model_params, dataset_params)
    
    def forward(self, x):
        self.Reservoir_State_Matrix = self.ESN(x)
        # print(f"resever state matrix {self.Reservoir_State_Matrix.shape}")
        self.ReadOut_Reservoir = self.ReadOut(self.Reservoir_State_Matrix)
        # print("重み:", self.ReadOut.readout_dense.weight)
        # print(self.ReadOut_Reservoir)

        #channle_size次元はいらないので，減らす
        self.ReadOut_Reservoir = self.ReadOut_Reservoir.squeeze(1)
        # print(f"output matrix {self.ReadOut_Reservoir.shape}")

        return self.ReadOut_Reservoir

    def reset_hidden_state(self):
        self.ESN.reset_hidden_state()

In [21]:
def make_time_coded_x(B, C, T, F, device):
    x = torch.zeros(B, C, T, F, device=device)
    t = torch.arange(T, device=device).view(1,1,T,1)
    print(t)
    x[..., 0:1] = t  # feature0に時刻を埋め込む
    print(x)
    return x

@torch.no_grad()
def debug_time_axis(model, B=2, C=1, T=12, F=4):
    x = make_time_coded_x(B,C,T,F,device)
    # forward前の確認
    for t in range(T):
        assert torch.all(x[:, :, t, 0] == t), "入力のtime軸が想定と違う"
    y = model.ESN(x)   # Reservoir states: (B,C,T,R)
    y_out = model(x)   # Reservoir states: (B,C,T,R)
    print(y.shape)
    print(y_out.shape)
    assert y.shape[2] == T, "Reservoir側でtime次元が崩れている"

dataset_params = {"seq_start" : 400, "seq_end" : 1200, "sequence_length": 800, "slicing_size" : 1, "augmentation_factor": 0, "batch_size" : 32, "Onehot_Encoding" : None, "augmentation_mu" : 0, "augmentation_sigma" : 0, "augmentation_shift" : 1, "time_stride":1}
model_params = {"reservoir_size" : 100,"input_size" : 4, "channel_size" : 1,  "reservoir_weights_scale" : 1, "input_weights_scale" : 10000, "spectral_radius" : 1.5,"reservoir_density" : 0.02, "leak_rate" : 0.1, "Batch_Training" : True, "ReadOut_output_size" : 25, "Regularization_L2" : 0.01}
training_params = {"num_epochs" : 1, "learning_rate" : 0.01, "weight_decay" : 1e-2, "testdata_ratio" : 0, "n_splits" : 5}

model = ESN(model_params, training_params, dataset_params).to(device)
debug_time_axis(model)

tensor([[[[ 0],
          [ 1],
          [ 2],
          [ 3],
          [ 4],
          [ 5],
          [ 6],
          [ 7],
          [ 8],
          [ 9],
          [10],
          [11]]]])
tensor([[[[ 0.,  0.,  0.,  0.],
          [ 1.,  0.,  0.,  0.],
          [ 2.,  0.,  0.,  0.],
          [ 3.,  0.,  0.,  0.],
          [ 4.,  0.,  0.,  0.],
          [ 5.,  0.,  0.,  0.],
          [ 6.,  0.,  0.,  0.],
          [ 7.,  0.,  0.,  0.],
          [ 8.,  0.,  0.,  0.],
          [ 9.,  0.,  0.,  0.],
          [10.,  0.,  0.,  0.],
          [11.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.],
          [ 1.,  0.,  0.,  0.],
          [ 2.,  0.,  0.,  0.],
          [ 3.,  0.,  0.,  0.],
          [ 4.,  0.,  0.,  0.],
          [ 5.,  0.,  0.,  0.],
          [ 6.,  0.,  0.,  0.],
          [ 7.,  0.,  0.,  0.],
          [ 8.,  0.,  0.,  0.],
          [ 9.,  0.,  0.,  0.],
          [10.,  0.,  0.,  0.],
          [11.,  0.,  0.,  0.]]]])
torch.Size([2, 1, 12, 100])
to